# Place for experimenting the progressive design

In [1]:
import os,sys
import yaml
import inspect
import importlib

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.gab_composer import GABTree,ROOT_UNIT_TEMPLATE,GABUnit
# from model_discovery.evolution import  BuildEvolution

ckpt_dir = os.environ['CKPT_DIR']
db_dir = U.pjoin(ckpt_dir, 'test_composer', 'db')
test_tree = GABTree('TestTree', db_dir)

prompts_dir='../model_discovery/agents/prompts/'
dir_prompt = U.pjoin(prompts_dir, 'prompts.yaml')
with open(dir_prompt, 'r') as file:
   gu_prompts = yaml.safe_load(file)
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gu_template.py'))
GAB_UNIT=inspect.getsource(GABUnit)

/media/8TBNVME/home/junyanc/model_discovery/notebooks/../model_discovery/model/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/media/8TBNVME/home/junyanc/model_discovery/notebooks/../model_discovery/model/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Failed to login to HuggingFace Hub, some datasets may not be available to download.


In [4]:
test_tree.path
test_tree.get_source('TestTree')

'# TestTree.py\n\nimport torch\nimport torch.nn as nn\n\nfrom model_discovery.model.utils.modules import GABUnit # DO NOT CHANGE THIS IMPORT STATEMENT #\n\n\n# YOU CAN IMPORT MORE MODULES HERE #\n\n# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #\n\n\nclass TestTree(GABUnit): \n    """Generalized Autoregressive Block\n        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}\n        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}\n        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable\n    """\n    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #\n        # argv: list of hyperparameters\n        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers\n        super().__init__(embed_di

In [11]:
import model_discovery.agents.prompts.prompts as P
importlib.reload(P)

gu_system_prompt=P.GU_DESIGNER_SYSTEM.format(GAB_BASE=P.GAB_BASE,GAM_PY=gam_py)
start_root_prompt=P.GU_DESIGN_SCRATCH_RAW.format(GAB_UNIT=GAB_UNIT,GU_TEMPLATE=GU_TEMPLATE,ROOT_UNIT_TEMPLATE=ROOT_UNIT_TEMPLATE)

print(start_root_prompt)



**Task:** You are designing an autoregressive model block. Due to its
complexity, we will break it into smaller components called **Generalized
Autoregressive Block Units (GABUnits)**. 

Each **GABUnit**: - Accepts a sequence of embeddings `X` and a dictionary of
intermediate variables `Z`. - Outputs a sequence of embeddings `Y` and an
updated dictionary `Z_`. - If `Z_` is provided, it should be merged into `Z` via
`Z.update(Z_)`.

**Structure:** - A **GABUnit** inherits from `nn.Module` as follows:

```python class GABUnit(nn.Module): 
    """ 
    Instead of directly giving the full implementation of a GAB block, the agent need to 
    design a series of nested GAB units and construct the full GAB block as a pipeline of these units.

    GAB is fractal, like GAB itself, each GAB unit accepts X and Z as input and returns Y and Z as output.
    """ 
    def __init__(self, embed_dim: int):
        super().__init__()
        self.embed_dim = embed_dim

    def _forward(self, X, **Z):
  